In [4]:
# Import required libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# Load data from a file
def load_data(file_path):
    with open(file_path, "r") as file:
        return file.readlines()

# Load datasets
train_file = "C:/Users/Prakash/OneDrive/Desktop/Project/emotion/train.txt"
val_file = "C:/Users/Prakash/OneDrive/Desktop/Project/emotion/val.txt"
test_file = "C:/Users/Prakash/OneDrive/Desktop/Project/emotion/test.txt"
train = load_data(train_file)
val = load_data(val_file)
test = load_data(test_file)

In [9]:
# Split data into tweets and labels
def get_tweet(data):
    tweets = []
    labels = []
    for line in data:
        tweet, label = line.strip().split(';')
        tweets.append(tweet)
        labels.append(label)
    return tweets, labels

In [10]:
# Extract tweets and labels
tweets, labels = get_tweet(train)
val_tweets, val_labels = get_tweet(val)
test_tweets, test_labels = get_tweet(test)

In [11]:
# Tokenize tweets
def get_sequences(tokenizer, tweets, maxlen=50):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded = pad_sequences(sequences, truncating='post', padding='post', maxlen=maxlen)
    return padded

In [12]:
# Tokenize tweets
tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(tweets)
padded_train_seq = get_sequences(tokenizer, tweets)
padded_val_seq = get_sequences(tokenizer, val_tweets)
padded_test_seq = get_sequences(tokenizer, test_tweets)

In [13]:
# Create label index mappings
classes = set(labels)
class_to_index = {c: i for i, c in enumerate(classes)}
index_to_class = {i: c for c, i in class_to_index.items()}
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])

In [14]:
# Convert labels to indices
train_labels = names_to_ids(labels)
val_labels = names_to_ids(val_labels)
test_labels = names_to_ids(test_labels)

In [15]:
# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000, 16),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
    tf.keras.layers.Dense(len(classes), activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
# Train the model
h = model.fit(padded_train_seq, train_labels, validation_data=(padded_val_seq, val_labels), epochs=20)

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 54s 81ms/step - accuracy: 0.3537 - loss: 1.5590 - val_accuracy: 0.6500 - val_loss: 0.9772
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 39s 78ms/step - accuracy: 0.7244 - loss: 0.7591 - val_accuracy: 0.7925 - val_loss: 0.6117
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 38s 76ms/step - accuracy: 0.8714 - loss: 0.3854 - val_accuracy: 0.8535 - val_loss: 0.4548
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 39s 77ms/step - accuracy: 0.9209 - loss: 0.2371 - val_accuracy: 0.8730 - val_loss: 0.4022
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 39s 77ms/step - accuracy: 0.9500 - loss: 0.1590 - val_accuracy: 0.8900 - val_loss: 0.3582
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 41s 77ms/step - accuracy: 0.9675 - loss: 0.1072 - val_accuracy: 0.8985 - val_loss: 0.3516
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 39s 78ms/step - accuracy: 0.9758 - loss: 0.0790 - val_accuracy: 0.8950 - val_loss: 0.3701
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 39s 78ms/step - accuracy: 0.9776 - loss: 0.0697 - 

In [17]:
# Function to predict emotion from the command line
def predict_emotion():
    while True:
        sentence = input("Enter a sentence (0 to quit): ")
        if sentence == "0":
            break
        if sentence == "":
            print("Enter the Sentence..!!")
            continue
        seq = tokenizer.texts_to_sequences([sentence])
        seq = pad_sequences(seq, maxlen=50, padding='post')
        p = model.predict(seq)[0]
        pred_class = index_to_class[np.argmax(p).astype('uint8')].capitalize()
        print(f"Predicted emotion: {pred_class}")

In [ ]:
# Start the prediction loop
predict_emotion()

Enter a sentence (0 to quit):  i want sex now


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Predicted emotion: Sadness


Enter a sentence (0 to quit):  it was a wonderful journey with pwc launchpad event


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Predicted emotion: Fear


Enter a sentence (0 to quit):  it was a wonderful day


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Predicted emotion: Joy


Enter a sentence (0 to quit):  it was a wonderful journey with chandru


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicted emotion: Joy


Enter a sentence (0 to quit):  it was a wonderful journey with pwc launchpad event


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Predicted emotion: Fear
